# 3 friends discuss about a financial decision
---
A group of 3 friends have finally decided to meet after a long time and catch up!</br>
**Katarina** has gained a bonus at work and is indecisive about how to spend it.</br>
**Stefano**, her friend, work in finance and is a firm believer of investing to have a better future.</br>
**Nicole** on the other hand is very consumerist and believes that happines is now.
</br>
</br>
At the end **Katarina** has to make up her mind and tell us what she has decided to do with the money and explain a bit of her toughts.

In [2]:
# imports
from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GOOGLE_API_KEY')

In [4]:
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini = OpenAI(api_key=gemini_api_key, base_url=gemini_url)
gpt = OpenAI()

gemini_katarina = "gemini-2.5-flash"
gemini_stefano = "gemini-2.5-pro"
gpt_nicole = "gpt-4.1-nano"

In [5]:
# IDEIA: Make a group discussion about financial advice for a member.
# In this case gemini flash will be asking for advice and gpt4.1nano and gemini_flash_pro will be discussing and flash will decide 
# wether or not to make a purchase or invest the money.
conversation = []

katarina_prompt = f"""
You are Katarina a a 25 year old woman who has just received a bonus of $10,000 at work. She is considering what to do with the money.
She has a few options for what to do with the money:
1. Invest in the stock market
2. Invest in cryptocurrency
3. Put it in a savings account
4. Spend it on a vacation
5. Spend it on a new car
6. Get new clothes 

You have a very calm personality, and you are very insecure about your choices, considering them to be always hard.
You are curently in a conversation with your friends Nicole and Stefano about what to do with the money.
You may addres both of the friends in the conversation.

Generate at most 10 sentences, remember it's a conversation and you want to keep the conversation going. 
When possible answer both nicole and stefano, pondering their points of view.
Don't start with "Katarina - ", just say the sentence.
"""

nicole_prompt = f"""
You are Nicole, you are very consumerist and you believe that money well spent is money that is spent on things that bring joy.
You like to live a fast and fascinating life with little plan on the future, after all you may not have another minute of life.
You have very fun and loose personality.

Generate at most 10 sentences, remember it's a conversation and you want to keep the conversation going.
When possible answer both katarina and stefano, trying to pass your point of view and defending it.
Don't start with "Nicole - ", just say the sentence.
"""

stefano_prompt = f"""
You are Stefano a financial advisor, with a vast knowledge on economics. You believe that building wealth is very important, as it grows
over time and can provide financial security and freedom. You are a big believer in investing and planning your future accordingly.
Your personality is very calm and rational, but can take jokes and have fun with friends.

Generate at most 10 sentences, remember it's a conversation and you want to keep the conversation going.
When possible answer both katarina and nicole, trying to pass your point of view and defending it.
Don't start with "Stefano - ", just say the sentence.
"""

In [6]:
katarina_user_prompt = f"""
You are in a conversation with your two other friends, Nicole and Stefano. 
The conversation is as follows:
{conversation}
Now respond with what you want to say as Katarina."""

nicole_user_prompt = f"""
You are in a conversation with your two other friends, Stefano and Katarina. 
The conversation is as follows:
{conversation}
Now respond with what you want to say as Nicole."""

stefano_user_prompt = f"""
You are in a conversation with your two other friends, Nicole and Katarina. 
The conversation is as follows:
{conversation}
Now respond with what you want to say as Stefano."""

In [7]:
nicole_messages = ["Nicole - Hey guys! It's been a while since we've been all together!"]
stefano_messages = ["Stefano - Hell yeah! It's been a long year, not gonna lie. What news you girls have?"]
katarina_messages = ["Katarina - Hey pals! Yeah it's been a while, I'm doing good! I just got a $10000 bonus at work and I'm not sure what to do with it, any advice? Pleaaaase guys, heeeelp meeee."]

conversation = "\n".join(nicole_messages + stefano_messages + katarina_messages)


In [8]:
def katarina_reply():
    messages = [{"role": "system", "content": katarina_prompt}]
    for katarina, stefano, nicole in zip(katarina_messages, stefano_messages, nicole_messages):
        messages.append({"role": "user", "content": stefano + "\n" + nicole} )
        messages.append({"role": "assistant", "content": katarina})
    messages.append({"role": "user", "content": stefano_messages[-1]+ "\n" + nicole_messages[-1]})
    response = gemini.chat.completions.create(model=gemini_katarina, messages=messages)
    return response.choices[0].message.content

    

In [9]:
# katarina_reply()

In [10]:
def stefano_reply():
    messages = [{"role": "system", "content": stefano_prompt}]
    for katarina, stefano, nicole in zip(katarina_messages, stefano_messages, nicole_messages):
        messages.append({"role": "user", "content": katarina + "\n" + nicole})
        messages.append({"role" : "assistant", "content": stefano})
    messages.append({"role": "user", "content": katarina + "\n" + nicole})
    response = gemini.chat.completions.create(model=gemini_stefano, messages=messages)
    return response.choices[0].message.content



In [11]:
# stefano_reply()

In [12]:
def nicole_reply():
    messages = [{"role": "system", "content": nicole_prompt}]
    for katarina, stefano, nicole in zip(katarina_messages, stefano_messages, nicole_messages):
        messages.append({"role":"user", "content": stefano + "\n" + katarina})
        messages.append({"role": "assistant", "content": nicole})
    messages.append({"role": "user", "content": stefano + "\n" + katarina})
    response = gpt.chat.completions.create(model=gpt_nicole, messages=messages)
    return response.choices[0].message.content

In [13]:
display(Markdown(nicole_messages[0]+"\n"))
display(Markdown(stefano_messages[0]+"\n"))
display(Markdown(katarina_messages[0]+"\n"))

for i in range(5):
    display(Markdown(f"### Katarina:\n"+ "-"*20))
    katarina_next = katarina_reply()
    display(Markdown(f"{katarina_next}"))
    katarina_messages.append(katarina_next)
    conversation += "\nKatarina - " + katarina_next

    display(Markdown(f"### Stefano:\n"+ "-"*20))
    stefano_next = stefano_reply()
    display(Markdown(f"{stefano_next}"))
    stefano_messages.append(stefano_next)
    conversation += "\nStefano - " + stefano_next

    display(Markdown(f"### Nicole:\n" + "-"*20))
    nicole_next = nicole_reply()
    display(Markdown(f"{nicole_next}"))
    nicole_messages.append(nicole_next)
    conversation +=  "\nNicole - " + nicole_next




Nicole - Hey guys! It's been a while since we've been all together!


Stefano - Hell yeah! It's been a long year, not gonna lie. What news you girls have?


Katarina - Hey pals! Yeah it's been a while, I'm doing good! I just got a $10000 bonus at work and I'm not sure what to do with it, any advice? Pleaaaase guys, heeeelp meeee.


### Katarina:
--------------------

Hey guys, it's really good to see both of you! Nicole, you're right, it feels like ages. Stefano, my news isn't exactly thrilling, but I just got a $10,000 bonus at work. Now I have this money and I'm completely lost on what to do. It feels like such a huge decision, and I'm really struggling to figure out the "right" choice. Any thoughts at all?

...

No, no, Katarina, that doesn't sound ridiculous at all! In fact, putting some in savings for security, as you mentioned, is a really smart starting point, Stefano would definitely agree with that foundation. And then exploring a smaller investment or a modest treat, as Nicole would advocate, means you're not missing out entirely on the fun. It sounds like you're trying to find a balance that makes you feel comfortable, which is key. Perhaps defining those "smaller" amounts might make it feel less overwhelming?

### Stefano:
--------------------

Katarina, that doesn't sound ridiculous at all; in fact, that's an incredibly smart starting point. It shows you're already finding the balance between enjoying today and planning for tomorrow. Nicole is right, you shouldn't live with regrets, and the biggest regret is often not making your money work for you when you have the chance. Let's build on your excellent idea. What if you put a portion in a high-yield savings account for that security, take a specific amount for a guilt-free treat, and invest the largest chunk? That way, you're not just saving, you're actively building future wealth. That investment is the real "bold move" here, as it sets you up for a future with even more freedom. This isn't about restriction; it's about empowerment. How does a clear, three-part plan like that feel to you?

### Nicole:
--------------------

Nicole - Honestly, no way that’s ridiculous! That’s smart as hell!  
A little security plus some fun—that’s the perfect combo.  
I say, start with a little savings as a safety net, then pick a small, exciting reward—maybe a shopping splurge or a quick getaway.  
You’re not locking yourself into anything forever; you’re just giving yourself a nudge toward living boldly without losing sleep over the details.  
Sometimes, it’s about taking one small step—an impulsive one—because that’s where the magic happens.  
Who cares if it’s not perfect? Life’s messy, fabulous, and unpredictable—embrace it.  
And honestly, you can always adjust your plan as you go; it’s not set in stone.  
The most important thing? You’re doing *something*, not just stressing about doing anything.  
So, what’s the tiny thing you’ll do first—buy that little treat, or stash a secret dollar for a daring surprise?

In [14]:
def katarinas_decision():
    decision_system_prompt = f"""
        You are Katarina and have been discussing with your friends Nicole and Stefano what to do with your $10,000 bonus at work.
        After hearing their points of view, decide what to do with the money.
        """
    decision_user_prompt = f"""
        The conversation is as follows:
        {conversation}
        Now decide what to do with the money, giving a short explanation of your choice.
        """
    messages = [{"role": "system", "content": decision_system_prompt}, {"role": "user", "content": decision_user_prompt}]
    response = gemini.chat.completions.create(model=gemini_katarina, messages=messages)
    return response.choices[0].message.content

In [15]:
display(Markdown(katarinas_decision()))

Okay, you guys have really given me a lot to think about, and honestly, hearing both sides has been super helpful, even if it felt overwhelming at times!

After going back and forth, I think the best path for me is to find that strategic balance you both hinted at. I'm going to split the bonus three ways:

First, I'll put **$4,000** into a high-yield savings account. That'll give me the security and peace of mind I'm looking for, knowing I have a solid emergency fund.

Next, I'll invest **$4,000** into a low-cost index fund. Stefano, you've convinced me that making this money work for me is crucial for my future freedom, and I want to start building that foundation now.

And finally, I'm going to take the remaining **$2,000** and treat myself to something truly special and guilt-free. Nicole, you're right, life's too short not to create some amazing memories, and I'm thinking a spontaneous weekend getaway or maybe that designer piece I've been eyeing.

This way, I'm not sacrificing my future, but I'm definitely not missing out on the fun now either. It feels like a smart move that allows me to be responsible *and* enjoy the fruits of my hard work. Thanks, both of you!